<a href="https://colab.research.google.com/github/sunspai1/ML_Code_Practise/blob/main/Build_a_CNN_architecture_using_Flowers_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
images_path = "/content/drive/MyDrive/flowers.zip"

In [3]:
!ls

drive  sample_data


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from zipfile import ZipFile
with ZipFile(images_path, 'r') as zip:
  zip.extractall()

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [7]:
pip install split-folders

In [8]:
import splitfolders
splitfolders.ratio('/content/flowers', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 4326 files [00:01, 3051.86 files/s]


In [13]:
# Initialising the CNN classifier
classifier = Sequential()


classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu', padding = 'same'))
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu', padding = 'same'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
classifier.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
classifier.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))


classifier.add(Flatten())


classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 128, activation = 'relu'))

# The final output layer with 5 neuron to predict the categorical classifcation
classifier.add(Dense(units = 5, activation = 'softmax'))

In [14]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
classifier.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory('/content/output/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


validation_set = test_datagen.flow_from_directory('/content/output/val',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')





Found 3457 images belonging to 5 classes.
Found 430 images belonging to 5 classes.


In [16]:
model = classifier.fit_generator(training_set,
                         
                         epochs = 100,
                         validation_data = validation_set,    
                         validation_steps = 2000)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
109/109 [==============================] - 17s 145ms/step - loss: 1.4062 - accuracy: 0.3656 - val_loss: 1.2124 - val_accuracy: 0.4279
Epoch 2/100
109/109 [==============================] - 14s 130ms/step - loss: 1.2332 - accuracy: 0.4620
Epoch 3/100
109/109 [==============================] - 14s 130ms/step - loss: 1.1412 - accuracy: 0.5221
Epoch 4/100
109/109 [==============================] - 15s 135ms/step - loss: 1.0716 - accuracy: 0.5820
Epoch 5/100
109/109 [==============================] - 15s 135ms/step - loss: 0.9850 - accuracy: 0.6240
Epoch 6/100
109/109 [==============================] - 15s 135ms/step - loss: 0.9059 - accuracy: 0.6575
Epoch 7/100
109/109 [==============================] - 15s 135ms/step - loss: 0.8901 - accuracy: 0.6650
Epoch 8/100
109/109 [==============================] - 14s 132ms/step - loss: 0.8033 - accuracy: 0.7012
Epoch 9/100
109/109 [==============================] - 15s 133ms/step - loss: 0.7912 - accuracy: 0.7058
Epoch 10/100
109/109 [

In [20]:
classifier.save('/content/flowers/classifier.h5')

classifier.save_weights('/content/flowers/classifier_weights.h5')

In [21]:
!ls

drive  flowers	output	sample_data


## **Check Model Performance on Test Data**

In [31]:
# Re-initalizing the test data generator with shuffle=False to create the confusion matrix
test_set = test_datagen.flow_from_directory('/content/output/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            shuffle=False,
                                            class_mode = 'categorical')

# Predict the whole generator to get predictions
Y_pred = classifier.predict_generator(test_set, int(500/32+1))

# Find out the predictions classes with maximum probability
y_pred = np.argmax(Y_pred, axis=1)

# Utilities for confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

# Printing the confusion matrix based on the actual data vs predicted data. 
print(confusion_matrix(test_set.classes, y_pred))

# Printing the classification report
print(classification_report(test_set.classes, y_pred, target_names=prediction_class))

Found 436 images belonging to 5 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


[[62  7  4  2  3]
 [14 81  0 10  1]
 [ 8  7 43  4 17]
 [ 1  4  1 64  4]
 [ 6  8 13  4 68]]
              precision    recall  f1-score   support

       daisy       0.68      0.79      0.73        78
   dandelion       0.76      0.76      0.76       106
        rose       0.70      0.54      0.61        79
   sunflower       0.76      0.86      0.81        74
       tulip       0.73      0.69      0.71        99

    accuracy                           0.73       436
   macro avg       0.73      0.73      0.73       436
weighted avg       0.73      0.73      0.73       436

